# 디렉토리 설정 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/DataSet/KYR_B_imaging.zip
!cp '/content/drive/MyDrive/Colab Notebooks/CT분류프로젝트/Custom_Py/init.py' ./

In [ ]:
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr3' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr2 74' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr5' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr1' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr7' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr8' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr4' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/nswr1' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr6' '/content/KYR_B_imaging' 
!rm -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱'

# 초기작업
- 데이터 제너레이터에 넣을 수 있는 기본형태를 만드는 것 

## 기본 DF 생성 
- csv 데이터인 snsb_df 
- 이미지 디렉토리 관련인 dir_df만듬 

전처리 과정에서 
- snsb_df -> csvs 
- dir_df -> new_dir_df로 변환 함 


In [1]:
import os 
import cv2 
import pandas as pd 
from glob import glob
from tqdm import tqdm
from init import init #초기 설정용 커스텀 패키지 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.pyplot as cm 
import warnings
warnings.filterwarnings('ignore')
from albumentations import (
    Rotate,Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate,JpegCompression
)

###snsb_df 생성 
- 환자 정보 데이터 프레임 

In [2]:
columns = ['성명',
 '병록번호',
 '만나이',
 '검사일',
 '교육년수',
 'SVLT_recall_total_score_z',
 'SVLT_Delayed_recall_z',
 'SVLT_recognition_score_z',
 'RCFT_immediate_recall_z',
 'RCFT_delayed_recall_z',
 'RCFT_recognition_score_z',
 'K_MMSE_total_score_z',
 'SNSB_II_Domain_Attention_z',
 'SNSB_II_Domain_Language_z',
 'SNSB_II_Domain_Visuospatial_z',
 'SNSB_II_Domain_Memory_z',
 'SNSB_II_Domain_Frontal_z']

snsb_df = pd.read_csv('/content/drive/MyDrive/DataSet/202202_김예림/SNSB_integerated.csv')
snsb_df = snsb_df.drop([2098,2591]) #병록번호 없는 행 제거 <- 결측치 
snsb_df['병록번호'] = snsb_df['병록번호'].apply(lambda x : str(int(x))) #소수점으로 되어 있는 것 처리하고 str형 변형 

snsb_df = snsb_df[columns]

###dir_df 생성 
- 환자  별 mri 디렉토리 데이터 프레임 
- 이미지 장수 정리 안된 데이터 프레임 

In [3]:
#폴더 디렉토리
folder_dir = glob('/content/KYR_B_imaging/*')
dir_df = init.dir_df(folder_dir)
dir_df = dir_df.sort_values(by=['key','dir'])
dir_df = dir_df.reset_index().drop(columns = 'index')

### 병록번호- key 값 보정
- 병록번호 == key 하게 만들어야 함 -> primary key 
- 중복 제거, @@@-1 형태 제거, 9자리로 통일 

In [4]:
target_length = [4,5,6,7,8] #<- 수정해야 하는 병록번호 갯수들 

dir_df,snsb_df = init.key_mismatch(dir_df,snsb_df,target_length)
#인덱스 초기화 
dir_df = dir_df.reset_index().drop(columns = 'index')
# plt.hist(snsb_df['병록번호'].map(len))
# plt.show()

#에러값들 제외 
errors = ['000758836-1' '030338456-1' '040145556-1' '050236926-1' '090374796-1'
 '100402746-1' '120010356-1' '870133669-1' '920033543-1' '930236559-1'
 '930257597-1' '970682600-1']

dir_df = dir_df.drop(np.where(dir_df['key'].map(len)>9)[0])

## new_dir_df 생성 - 이미지 장수 통일 
- 위에서 만든 dir_df에서 각 key 별로 이미지 19장으로 맞춤 
- 19장으로 맞추는 이유는 가장 비율이 높아서 


### key 값 별 이미지 장수 연산 

In [5]:
#이미지 장수 연산 
keys = list(set(dir_df['key'])) 
images_length = pd.DataFrame(keys)[0].apply(lambda x: init.check_images(dir_df,x)) #키 값별로 이미지의 장수들 계산 
# images_length.value_counts()

length_df = pd.DataFrame([keys,images_length]).T
length_df.columns = ['key','image_length']
length_df['image_length']  = length_df['image_length'].map(int)

#Standard를 기준으로 less over 분할 
standard = 19 # 통일 시킬 이미지 장 수 기준 

# 조정이 필요한 장 수 초과하는 것들만 추림 
length_df_over = length_df[length_df['image_length']>standard]

#19장 보다 적은 경우 
# length_df_less = length_df[length_df['image_length']<standard]

len(length_df_over)

1606

### key값 별로 이미지 19장 있는 new_dir_df 생성 

In [6]:
new_dir_df = pd.DataFrame(np.zeros(len(length_df_over)*19*2).reshape(-1,2))
new_dir_df.columns = dir_df.columns

for index in tqdm(range(len(length_df_over))):
  new_dir_df.iloc[index*19:(index+1)*19,:]  = dir_df.loc[init.droped_indexes(dir_df,length_df_over,index,standard=19)]
new_dir_df = new_dir_df.sort_values(by=['key','dir'])
new_dir_df = new_dir_df.reset_index().drop(columns = 'index')

100%|██████████| 1606/1606 [00:07<00:00, 216.48it/s]


## csvs 생성 
 - 중복 제거 
 - z score -1.0를 기준으로 0(정상) 1(비정상)으로 분류 



In [7]:
#19장의 사진으로 추린 new_dir_df로 새로운 키 리스트만듬) 

key_lists = np.unique(new_dir_df['key'])
key_lists.sort()

Embedding_columns = snsb_df.columns[5:]

for column in Embedding_columns:
  snsb_df[f'E_{column}'] = snsb_df[column].map(init.zscore_Embedding)

#정렬 
snsb_df = snsb_df.sort_values(by=['병록번호'])
new_dir_df = new_dir_df.sort_values(by=['key','dir'])

#snsb_df 중 key_lists에 있는 행들만 추림 -> csvs 
csvs = snsb_df[snsb_df['병록번호'].apply(lambda x : x in key_lists)].sort_values(by=['병록번호'])
csvs = csvs.reset_index().drop(columns = 'index')

#중복 제거 : 이름으로 정렬 후 중복 제거 
csvs = csvs.sort_values(by=['성명'])
csvs = csvs.drop_duplicates(['병록번호'], keep = 'first')

#다시 병록번호 로 정렬 해서 csvs 와 new_dir_df 순을 맞춤 
csvs = csvs.sort_values(['병록번호'])
csvs = csvs.reset_index().drop(columns = 'index')
new_dir_df = new_dir_df.sort_values(by=['key','dir'])
new_dir_df = new_dir_df.reset_index().drop(columns = 'index')

#맞춤 확인 
print(len(csvs),len(np.unique(new_dir_df['key'])))

1606 1606


## 결측치 처리 

In [ ]:
#컬럼 중 isnull 검색 
#검색한 표본들의 평균 나이 
#평균 나이로 원래  csvs 검색 
#해당 나이로 검색 된 것들의 평균 값으로 insull 채움 

In [9]:
null_columns = ['RCFT_immediate_recall_z',         
'RCFT_delayed_recall_z',            
'RCFT_recognition_score_z',        
'K_MMSE_total_score_z',           
'SNSB_II_Domain_Attention_z',       
'SNSB_II_Domain_Language_z',        
'SNSB_II_Domain_Visuospatial_z',    
'SNSB_II_Domain_Memory_z',          
'SNSB_II_Domain_Frontal_z']         

In [10]:
for null_column in null_columns:
  try:
    null_age = int(np.mean(csvs[csvs[null_column].isnull()]['만나이']))
    null_value = np.mean(csvs[csvs['만나이']==null_age][null_column].dropna())
    csvs.loc[csvs[null_column].isnull(),null_column] = null_value
  except:
    pass

In [11]:
csvs.isnull().sum().sum()

0

# 전처리

## Augmentation 

In [12]:
#이미지 콘트라스트 변경 
def img_Contrast(img): 
  import cv2 
  lab = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
  lab = cv2.cvtColor(lab, cv2.COLOR_BGR2LAB)
  l, a, b = cv2.split(lab)
  clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8)) 
  cl = clahe.apply(l)
  limg = cv2.merge((cl, a, b))
  final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
  return final 

#가우시안 블러 
def gaussianblur(image):
  import cv2 
  import numpy as np 
  blur = cv2.GaussianBlur(image, (5,5), np.random.randint(0,3,1))
  return blur


import albumentations as A
transform = A.Compose([
    RandomBrightness(limit=0.1),
    JpegCompression(quality_lower=85, 
                    quality_upper=100, p=0.5), 
    HueSaturationValue(hue_shift_limit=20,
                        sat_shift_limit=30, 
                    val_shift_limit=20, p=0.5),
    RandomContrast(limit=0.2, p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
])


#랜덤 어그먼테이션 
def random_aug(aug_image):
  import numpy as np 
  import cv2 
  n = np.random.randint(0,3,1)
  if n == 0:
    aug_image = cv2.flip(aug_image,np.random.randint(0,2,1))
  if n == 1:
    aug_image = gaussianblur(aug_image)
  if n== 2:
    pass
  return aug_image


## 이미지 전처리

In [55]:
#이미지 전처리 

def img_preprocess(img):
  shape = img.shape[0]
  preprocessed_img = img[int(shape*0.05):int(shape*0.95),int(shape*0.1):int(shape*0.9),:] #crop 
  preprocessed_img = transform(image=preprocessed_img)['image']
  # preprocessed_img = cv2.cvtColor(preprocessed_img,cv2.COLOR_BGR2GRAY)
  preprocessed_img = cv2.resize(preprocessed_img,dsize=(IMAGE_SIZE,IMAGE_SIZE)) #resize Albu mentations -> vision 관련 augmentation document 
  preprocessed_img = preprocessed_img/255.
  return preprocessed_img 


def key_to_dir(key,new_dir_df):
  dir_of_key = new_dir_df[new_dir_df['key']==key]
  return dir_of_key



def read_imgs(img_dir_df): #224, 224, 19 용 
  temp_imgs = np.zeros(IMAGE_SIZE*IMAGE_SIZE*IMAGE_CHANNELS*3).reshape(IMAGE_CHANNELS,IMAGE_SIZE,IMAGE_SIZE,3)
  for i in range(len(img_dir_df)):
    temp_img = img_dir_df['dir'].iloc[i]
    temp_img = img_preprocess(cv2.imread(temp_img))
    temp_imgs[i,:,:,:] = temp_img 
  return temp_imgs



# 데이터 제너레이터

## 사전설정

In [179]:
input_columns = ['만나이','교육년수']
output_columns = ['E_SVLT_recall_total_score_z', 'E_SVLT_Delayed_recall_z',
       'E_SVLT_recognition_score_z', 'E_RCFT_immediate_recall_z',
       'E_RCFT_delayed_recall_z', 'E_RCFT_recognition_score_z',
       'E_K_MMSE_total_score_z', 'E_SNSB_II_Domain_Attention_z',
       'E_SNSB_II_Domain_Language_z', 'E_SNSB_II_Domain_Visuospatial_z',
       'E_SNSB_II_Domain_Memory_z', 'E_SNSB_II_Domain_Frontal_z']
output_column = output_columns[0]

csvs_max = np.array([90,18])
csvs_min = np.array([45,0])
IMAGE_SIZE = 128
IMAGE_CHANNELS = 19 
BATCH_SIZE = 16 

## !!!제너레이터

키 리스트를 이용해 train, valid, test 분리 

In [216]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import Sequence
import math


class DataGenerator(Sequence):
  def __init__(self,
                 new_dir_df,
                 csvs,
                 key_lists,
                 output_columns,
                 batch_size: int,
                 shuffle: bool = False,
                 rescale:bool = True) -> None:
        self.input_columns = input_columns
        self.output_columns = output_columns
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.new_dir_df = new_dir_df
        self.csvs = csvs
        self.key_lists = key_lists

  def __len__(self):
    return math.ceil(len(self.key_lists) / self.batch_size)
    
  def on_epoch_end(self):
    self.indices = np.arange(len(self.key_lists))
    if self.shuffle == True:
      np.random.shuffle(self.indices,)
      self.key_lists = self.key_lists[self.indices]
      

  def __getitem__(self, index):
    self.batch_keys = self.key_lists[index*self.batch_size : (index+1)*self.batch_size]

    #imgs 
    temp_imgs = np.arange(16*IMAGE_CHANNELS*IMAGE_SIZE*IMAGE_SIZE*3).reshape(16,IMAGE_CHANNELS,IMAGE_SIZE,IMAGE_SIZE,3)
    for key in range(len(self.batch_keys)):
      img_dir_df = key_to_dir(self.batch_keys[key],self.new_dir_df)
      temp_img = read_imgs(img_dir_df)
      temp_imgs[key,:,:,:] = temp_img
    input_imgs = np.array(temp_imgs).astype(np.float)

    #input_csvs
    temp_csvs = self.csvs[self.csvs['병록번호'].apply(lambda x: x in self.batch_keys)]
    temp_csvs = temp_csvs.sort_values(by=['병록번호'])
    temp_csvs = temp_csvs.reset_index().drop(columns='index')
    input_csvs = temp_csvs[input_columns]
    input_csvs = (np.array(input_csvs)-csvs_min)/(csvs_max-csvs_min)
    input_csvs = np.array(input_csvs).astype(np.float)

    #output_csvs
    temp_csvs = self.csvs[self.csvs['병록번호'].apply(lambda x: x in self.batch_keys)]
    temp_csvs = temp_csvs.sort_values(by=['병록번호'])
    temp_csvs = temp_csvs.reset_index().drop(columns='index')
    output_csvs = temp_csvs[self.output_columns]
    output_csvs = np.array(output_csvs).astype(np.float)

    # return [input_imgs,input_csvs],output_csvs
    return input_imgs,output_csvs

    


#모델

In [180]:
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import CosineDecay, ExponentialDecay
from tensorflow.keras.metrics import Recall
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD 
from tensorflow.keras import layers
from tensorflow.keras.layers import Bidirectional, LSTM, GRU, Activation, AveragePooling2D,ConvLSTM1D
from tensorflow.keras.layers import BatchNormalization,Conv2D,MaxPooling2D,Activation,Dropout,Dense,Flatten,Input,Conv3D,MaxPooling3D
from tensorflow.keras.layers import Conv3D, ConvLSTM2D, BatchNormalization, Input, Dropout, Bidirectional, Conv1D, MaxPooling1D,Reshape
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.applications import resnet50, efficientnet,VGG19,EfficientNetV2S, NASNetLarge, ResNet50V2, vgg19,resnet_v2
from tensorflow.keras.applications import ResNet50, EfficientNetB0, DenseNet169, MobileNet,EfficientNetB4, ResNet152V2, EfficientNetV2B0
import math
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from keras.layers.pooling import GlobalAveragePooling2D,GlobalAveragePooling3D,AveragePooling3D
from tensorflow.keras.utils import plot_model

## !!!모델 사전 정의

In [171]:
res50 = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE,3)) 
res50.trainable = False

vgg_19 = VGG19(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE,3))
vgg_19.trainable = False

effv2 = EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE,3)) 
effv2.trainable = False

In [71]:
def input_to_pre(image_input): 
  x = Conv2D(16, (2,2), padding ='same',activation = 'relu')(image_input)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)
  # x = Conv2D(8, (2,2), padding ='same',activation = 'relu')(x)
  # x = BatchNormalization()(x)
  # x = Dropout(0.2)(x)
  out = Conv2D(3, (2,2), padding ='same',activation = 'relu')(x)
  return out 

def image_encoder_layer(filters,img_input):
  img = Conv2D(filters,(3,3),padding='same')(img_input)
  img = Dropout(0.2)(img)
  img_output = BatchNormalization()(img)
  return img_output


def image_encoder(image_combined):
  # img = image_encoder_layer(1024,image_combined)
  # img = image_encoder_layer(512,image_combined)
  # img = image_encoder_layer(256,image_combined)
  img_out = image_encoder_layer(128,image_combined)
  return img_out

def csvs_layers(filters,csv_input):
  csv = Dense(filters, activation = 'relu')(csv_input)
  csv = Dropout(0.2)(csv)
  csv_layer_out = BatchNormalization()(csv)
  return csv_layer_out

def csvs_decoder(csvs_input):
  # csvs = csvs_layers(32,csvs_input)
  # csvs = csvs_layers(64,csvs)
  csvs_out = csvs_layers(128,csvs_input)
  return csvs_out
  


## 콜백

In [181]:
# copy from https://gist.github.com/stared/dfb4dfaf6d9a8501cd1cc8b8cb806d2e
# loss 실시간 출력 
from IPython.display import clear_output
from tensorflow.keras.callbacks import Callback

class PlotLosses(Callback):

  def on_train_begin(self, logs={}):
    self.epochs = []
    self.losses = []
    self.val_losses = []
    self.logs = []
    self.fig = plt.figure()


  def on_epoch_end(self, epoch, logs={}):

    self.epochs.append(epoch)
    self.losses.append(logs.get('loss'))
    self.val_losses.append(logs.get('val_loss'))

    clear_output(wait=True)
    plt.plot(self.epochs, self.losses, label="loss")
    plt.plot(self.epochs, self.val_losses, label="val_loss")
    plt.legend()
    plt.show();
    print("loss = ", self.losses[-1], ", val_loss = ", self.val_losses[-1])

#call backs 선언
plot_losses = PlotLosses()

from tensorflow.keras.callbacks import ModelCheckpoint #<- model 저장 객체, best_model.h5라는 모델 이 저장 됨 
model_check_point = ModelCheckpoint(
    'best_model.h5', 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True)


from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # 모니터링 대상, 이걸 기준으로 멈춤 
    verbose=1,
    patience=10)         # 중지까지의 여유분

from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', # 모니터링 대상
    patience=10,        # 대상ㅃ 기간동안 유지
    factor=0.2,         # 줄이는 양                              
    min_learning_rate=0.00001)     # 최소 학습율

callbacks = [ plot_losses]

## !!!3D - 모델 

In [214]:
def make_model():
  image_input = Input((19,IMAGE_SIZE,IMAGE_SIZE,3))
  image = Bidirectional(ConvLSTM2D(3,2,padding='same'))(image_input)
  image = BatchNormalization()(image)
  image = Dropout(0.2)(image)
  image = Conv2D(3, 3,padding = 'same')(image)
  image = res50(image)
  image = GlobalAveragePooling2D()(image)
  image = Dense(2, activation = 'softmax')(image)
  model = Model(image_input,image)
  return model


## !!!Compile

In [209]:
decay_steps = 1000
initial_learning_rate = 0.0005
lr_decayed_fn = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps)

In [210]:
optimizer = Adam(learning_rate = lr_decayed_fn)

In [183]:
from keras import backend as K
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

## !!!Cross Validation Train

In [211]:
def train_valid_index(mode):
  full_cv = [[0, 289], [289, 578], [578, 867], [867, 1156], [1156, 1445]]
  sample_cv = [[0, 60], [60, 120], [120, 180], [180, 240], [240, 300]]
  sample_key_lists = key_lists[:300]
  if mode == 'sample':
    cv = sample_cv
    keys = sample_key_lists
  if mode == 'full':
    cv = full_cv
    keys = key_lists
  return cv, keys

In [212]:
cv, keys =train_valid_index('sample')
i=0
train_index = np.delete(np.arange(len(keys)),[np.arange(cv[i][0],cv[i][1])])
valid_index = np.arange(cv[i][0],cv[i][1])
test_index = np.arange(1446,1606)

train_generator = DataGenerator(new_dir_df,
                                csvs,
                                keys[train_index],
                                output_columns[0],
                                batch_size=BATCH_SIZE,
                                shuffle=True)
valid_generator = DataGenerator(new_dir_df,
                                csvs,
                                keys[valid_index],
                                output_columns[0],
                                batch_size=BATCH_SIZE,
                                shuffle=False)
test_generator = DataGenerator(new_dir_df,
                               csvs,
                               key_lists[test_index],
                               output_columns[0],
                               batch_size=BATCH_SIZE,
                               shuffle=False)

In [ ]:
model_lists = []
history_lists = []
for i in range(2):
  train_index = np.delete(np.arange(len(keys)),[np.arange(cv[i][0],cv[i][1])])
  valid_index = np.arange(cv[i][0],cv[i][1])
  test_index = np.arange(1446,1606)

  train_generator = DataGenerator(new_dir_df,csvs,keys[train_index],output_columns[0],batch_size=BATCH_SIZE,shuffle=True)
  valid_generator = DataGenerator(new_dir_df,
                                csvs,
                                keys[valid_index],
                                output_columns[0],
                                batch_size=BATCH_SIZE,
                                shuffle=False)
  test_generator = DataGenerator(new_dir_df,csvs,key_lists[test_index],output_columns[0],batch_size=BATCH_SIZE,shuffle=False)
  
  model = make_model()
  model.compile(optimizer=optimizer,loss = 'sparse_categorical_crossentropy',metrics=['accuracy',recall,precision,f1score])
  history = model.fit(train_generator,validation_data = valid_generator, epochs=20,verbose=1,callbacks=callbacks)
  
  model_lists.append(model)
  history_lists.append(history)


In [ ]:
model.summary()

In [ ]:
plot_model(model)

kfold 진행 
- 컬럼별 분포 보고 stratified 하게 folding 
- 100 에폭 돌리지 말고 초반에 적당히 되면 스탑 
- nosiy student 
- 한개 모델로 하지말고 여러 모델 앙상블 
- 